# Building Queries for Snapshots

Designing a query is the most important part of data selection. It can have implications on how relevant the selected data is for the workflow.

To guide in this process, Snapshot operations use the `SnapshotQuery` class.

# SnapshotQuery class
A query is at its minimum a where statement. However, this class offers additional attributes to support specific operations like Analytics or Extraction. A detailed description of the syntax, fields and potential values can be seen in the [Data Selection Samples - Dow Jones Developer Portal](https://developer.dowjones.com/site/docs/getting_started/data_selection_samples/index.gsp).

* **`where`**: _Required_, _Positional_. String that specifies the Where statement as used in Google BigQuery SQL Sytnax.
* **`includes`**: _Optional_, _Default: `None`_. Object-type syntax that specifies the list of fields and values to include in bulk. Value can be set as ```{"company_codes": ['mcrost', 'applc'] }```
* **`excludes`**: _Optional_, _Default: `None`_. Object-type syntax that specifies the list of fields and values to exclude in bulk. Value can be set as ```{"subject_codes": ['gspo', 'nnam'] }```
* **`frequency`**: _Optional_, _Default: `'MONTH'`_. Frequency to aggregate results. Possible values are `'MONTH'`, `'YEAR'` or `'DAY'`. Used only by the Analytics operation.
* **`date_field`**: _Optional_, _Default: `'publication_datetime'`_. Date-time field used to aggregate. Possible values are `'publication_datetime'`, `'ingestion_datetime'` and `'modification_datetime'`. Used only by the Analytics operation.
* **`group_by_source_code`**: _Optional_, _Default: `False`_. Specifies if the results will contain only a year field (`False`), or if each year will be broken-down by source (`True`). Used only by the Analytics operation.
* **`top`**: Valid only if ```group_by_source_code``` is True. Limits the number of sources used to group the content by frequency. Used only by the Analytics operation.
* **`file_format`**: _Optional_, _Default: `'avro'`_. File format to be used for Extractions. Possible values are `'avro'`, `'csv'` or `'json'`. Used only by the Extraction operation.
* **`limit`**: _Optional_, _Default: `0` (No limit)_. Positive integer that limits the amount of documents to extract. Used only by the Extraction operation.

This class is described in more detail in the [package documentation](https://factiva-news-python.readthedocs.io/en/latest/factiva.news/snapshot.html#snapshot-query).

# Query Snippets

## Where Attribute
The where attribute is the most used attribute to select content by applying different conditions to the available fields. Below there are some code snippets that help building a where statement, and notes that help modify the syntax according to the need.

### Range of dates
It works with the standard SQL syntax. If updates will be collected in the future, avoid using an end-date.
Alternative fields are: `modification_datetime` and `ingestion_datetime`.
```sql
publication_datetime >= '2010-01-01 00:00:00' AND publication_datetime <= '2020-06-30 23:59:59' "
```

### Filter by `source_codes`
The following clause is useful to select sources by their individual code.
```sql
UPPER(source_code) IN ('AASFNE', 'HTACCF', 'NLADLW', 'ADVTSR', 'AFNROL', 'AGEEOL', 'AGEE', 'HNASNI', 'APRS', 'ASXTEX', 'AUSTOL')
```
In case sources will be selected by their category or source family, a better option is using `restrictor_codes`. This field is not in the documentation, but the CSE or Integration Team can provide more information like source family codes
```sql
REGEXP_CONTAINS(restrictor_codes, r'(?i)(^|,)(sfwsj|sflefig|sfechos|sfnyt)($|,)')
```

### Filter by `subject_codes`
```sql
REGEXP_CONTAINS(subject_codes, r'(?i)(^|,)(mcat|ccat|ecat|gglobe|ghea|ghnwi|gcns|gpir|gdatap|greest|grisk|gsci|gspace|gtrans)($|,)')
```

### Filtering by `language_code`
```sql
LOWER(language_code) IN ('en', 'es', 'it')
```

### Filtering by `company_codes_xxxx`
This is applicable to any company-related fields (about, occur or company_codes and other combinations with identifiers - ISIN, CUSIP...).
- company_codes
- company_codes_about
- company_codes_occur
- ...
- company_codes_about_isin
- ...

#### Short list of codes
```sql
REGEXP_CONTAINS(company_codes, r'(?i)(^|,)(agbpet|agip|agphng|agpnme|agzgi|altgaz|bbor|brnene|distrg|eenivm|egapg|enichm|enie|enimnt)($|,)')
```

#### At least 1 code
In case the interest is to ensure at least one company is tagged (the field is not empty), the expressions looks like this.
```sql
LENGTH(company_codes) > 2
```

#### Exactly X codes
When searching for an exact number of tagged companies, it's necessary to count the number of entries in the resulting array after splitting the string.
```sql
ARRAY_LENGTH(SPLIT(company_codes_about, ',')) = 3
```
Use `n+2` depending on the `n` companies looking to be tagged (3 for 1 code, 4 for 2 codes, 5 for 3 codes, and so on).  This example selects articles with exactly 1 company tagged as `company_codes_about`.

#### Long list of codes
When having a long list of FCodes in an external file like CSV, adding all values to the regex expression requires the use of programming code. In this case this shows how to do it in Python.
```python
import pandas as pd

comp_list = pd.read_csv('fcodes_list.csv')
comp_list_regex = ""

for index, fcode in comp_list['fcode'].items():
    if index == 0:
        comp_list_regex = f"{fcode}"
    else:
        comp_list_regex = f"{comp_list_regex}|{fcode}"

where_statement = f"REGEXP_CONTAINS(company_codes_about, r'(?i)(^|,)({comp_list_regex})($|,)')"

```

### Filtering by `industry_codes`
```sql
REGEXP_CONTAINS(industry_codes, r'(?i)(^|,)(i1|i25121|i2567)($|,)')
```

### Filtering by `person_codes`
```sql
REGEXP_CONTAINS(LOWER(person_codes), r'(?i)(^|,)(76064380|2349856)($|,)
```

### Filtering by the **region** the article is **about** (`region_codes`)
```sql
REGEXP_CONTAINS(region_codes, r'(?i)(^|,)(aust|spain|italy|usa|uk)($|,)')
```

### Filtering by the **region** where the source is **headquarted** (`region_of_origin`)
```sql
REGEXP_CONTAINS(region_of_origin, r'(?i)(aust|spain|italy|usa|uk)')
```

### Filtering by the number of words (`word_count`)
```sql
word_count >= 250
```

### Filtering by terms in full-text (Keyword search)
```sql
REGEXP_CONTAINS(CONCAT(title, ' ', IFNULL(snippet, ''), ' ', IFNULL(body, '')), r'(?i)(^|\b)(economic|economy|regulation|deficit|budget\W+tax|central\W+bank)($|.|\b)')
```

More examples are available in the Data Selection Samples in the Dow Jones Developer Portal (https://developer.dowjones.com/site/docs/getting_started/data_selection_samples/index.gsp#)


Building the where statement. Python concatenates the strings when inside the parenthesis. Mind the extra space at the end of each string.

In [1]:
where_statement =  (
                    r" publication_datetime >= '2020-01-01 00:00:00' "
                    r" AND publication_datetime <= '2021-04-30 23:59:59' "
                    r" AND REGEXP_CONTAINS(CONCAT(title, ' ', IFNULL(snippet, ''), ' ', IFNULL(body, '')), r'(?i)(^|\b)(economic|economy|regulation|deficit|budget\W+tax|central\W+bank)($|.|\b)') "
                    r" AND REGEXP_CONTAINS(subject_codes, r'(?i)(^|,)(cprdcl|c16|cbro)($|,)') "
                    r" AND REGEXP_CONTAINS(region_codes, r'(?i)(^|,)(spain|italy)($|,)') "
 )